# Import

In [11]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

## Add configuration file

In [12]:
sys.path.append("/home/jovyan/core/config/")

In [13]:
from ALL import config 

## Set condition

In [14]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [15]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [16]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [17]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["gmm"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]
covariance_types = config["clustering"]["gmm"]["covariance_types"]

In [18]:
n_components = 4

In [19]:
def getGMM(vectors, n_components, covariance_type, seed, path):
    gmm = GaussianMixture(
        n_components=n_components, covariance_type=covariance_type, random_state=seed
    )
    gmm.fit(vectors)
    # save model
    os.makedirs(os.path.dirname(path), exist_ok=True)
    pickle.dump(gmm, open(path, "wb"))
    pred = gmm.predict(vectors)
    return pred

In [20]:
vectors_path = "../../data/AgNews/doc2vec/vector/"
models_path = "../../data/AgNews/doc2vec/GMM/model/"
pred_path = "../../data/AgNews/doc2vec/GMM/pred/"
for vector_dim in tqdm(vector_dims):
    for model_num in range(model_nums):
        for covariance_type in covariance_types:
            vector_path = (
                f"{vectors_path}{vector_dim}/normalized/{vector_model_num}.csv"
            )
            vectors = np.loadtxt(vector_path, delimiter=",")

            pred = getGMM(
                vectors,
                seed=model_num,
                n_components=n_components,
                covariance_type=covariance_type,
                path=f"{models_path}{vector_dim}/{covariance_type}/{model_num}.sav",
            )

            # save prediction
            os.makedirs(f"{pred_path}{vector_dim}/{covariance_type}", exist_ok=True)
            with open(
                f"{pred_path}{vector_dim}/{covariance_type}/{model_num}.csv", "w"
            ) as f:
                writer = csv.writer(f)
                writer.writerow(pred)

 86%|████████▌ | 12/14 [20:41<09:09, 274.66s/it]/opt/conda/lib/python3.10/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
 93%|█████████▎| 13/14 [39:11<08:47, 527.76s/it]/opt/conda/lib/python3.10/site-packages/sklearn/mixture/_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
100%|██████████| 14/14 [1:24:24<00:00, 361.75s/it] 
